# Gemini API: Embeddings Quickstart

The Gemini API generates state-of-the-art text embeddings. An embedding is a list of floating point numbers that represent the meaning of a word, sentence, or paragraph. You can use embeddings in many downstream applications like document search.

This notebook provides quick code examples that show you how to get started generating embeddings.


## Setup

### Install the Google GenAI SDK

Install the Google GenAI SDK from [npm](https://www.npmjs.com/package/@google/genai). 

```bash
$ npm install @google/genai
```

### Setup your API key

You can [create](https://aistudio.google.com/app/apikey) your API key using Google AI Studio with a single click.

Remember to treat your API key like a password. Don't accidentally save it in a notebook or source file you later commit to GitHub. In this notebook we will be storing the API key in a `.env` file. You can also set it as an environment variable or use a secret manager. 

Here's how to set it up in a `.env` file:

```bash
$ touch .env
$ echo "GEMINI_API_KEY=<YOUR_API_KEY>" >> .env
```

:::{.callout-tip}

Another option is to set the API key as an environment variable. You can do this in your terminal with the following command:

```bash
$ export GEMINI_API_KEY="<YOUR_API_KEY>"
```
:::

### Load the API key

To load the API key from the `.env` file, we will use the `dotenv` package. This package loads environment variables from a `.env` file into `process.env`. 

```bash
$ npm install dotenv
```

Then, we can load the API key in our code:


In [2]:
const dotenv = require("dotenv") as typeof import("dotenv");

dotenv.config({
  path: "../.env",
});

const GEMINI_API_KEY = process.env.GEMINI_API_KEY ?? "";
if (!GEMINI_API_KEY) {
  throw new Error("GEMINI_API_KEY is not set in the environment variables");
}
console.log("GEMINI_API_KEY is set in the environment variables");


GEMINI_API_KEY is set in the environment variables


:::{.callout-note}
In our particular case the `.env` is is one directory up from the notebook, hence we need to use `../` to go up one directory. If the `.env` file is in the same directory as the notebook, you can omit it altogether. 

```
│
├── .env
└── quickstarts
    └── Embeddings.ipynb
```
:::


### Initialize SDK Client

With the new SDK, now you only need to initialize a client with you API key (or OAuth if using [Vertex AI](https://cloud.google.com/vertex-ai)). The model is now set in each call.


In [3]:
const google = require("@google/genai") as typeof import("@google/genai");

const ai = new google.GoogleGenAI({ apiKey: GEMINI_API_KEY });


### Select a model

Now select the model you want to use in this guide, since this an embeddings quickstart, we will use the `text-embeddings-004` model. Text embeddings are used to measure the relatedness of strings and are widely used in many AI applications.



In [4]:
const tslab = require("tslab") as typeof import("tslab");

const MODEL_ID = "text-embedding-004";


## Embed content

Call the `embedContent` method with the `models/text-embedding-004` model to generate text embeddings.


In [8]:
const text_embeddings = await ai.models.embedContent({
  model: MODEL_ID,
  contents: ["Hello world"],
});
console.log("Text Embeddings:", text_embeddings.embeddings?.[0].values?.slice(0, 50), "...TRIMMED");


Text Embeddings: [
    0.013168517,   -0.00871193,  -0.046782672,
  0.00069969177,  -0.009518872,  -0.008720178,
     0.06010358,   0.024755737,   0.026053527,
    0.054356426,   -0.03793384, -0.0014235445,
    0.030605137,  -0.015512642,  -0.012904964,
    -0.02880739,  -0.007819577,   0.012152762,
     -0.1139952,   0.010654231,   0.005365246,
   -0.001178891,  -0.029781109,  -0.060107403,
   -0.015272871, -0.0036046242,   0.006147686,
    0.031175768,   0.021421982,    0.03710434,
   -0.037202735,   0.046146937,   0.002196372,
   -0.031793054,   0.009660255,   0.012500477,
     -0.0509635,     0.0211728,   0.014332891,
   -0.057802226,  -0.027034516,    0.03680537,
   0.0016361808,     0.0085209,    0.04331588,
   -0.032519087,   0.018076202, -0.0031592466,
   0.0045996527, -0.0063372543
] ...TRIMMED


In [10]:
console.log("Text Embeddings Length:", text_embeddings.embeddings?.[0].values?.length);


Text Embeddings Length: 768


## Batch embed content

You can embed a list of multiple prompts with one API call for efficiency.


In [11]:
const batch_embeddings = await ai.models.embedContent({
  model: MODEL_ID,
  contents: ["What is the meaning of life?", "How much wood would a woodchuck chuck?", "How does the brain work?"],
});
console.log(
  "Batch Text Embeddings:",
  batch_embeddings.embeddings?.map((e) => e.values?.slice(0, 50)),
  "...TRIMMED"
);


Batch Text Embeddings: [
  [
     -0.010632273,  0.019375853,    0.020965198,
     0.0007706437, -0.061464068,    0.014739741,
    -0.0022759985,  0.013184195,    0.014464715,
      0.022593116,   0.02184836,   -0.059616957,
       0.06032222, -0.047657482,    0.017848385,
      -0.10987464,   -0.0598155,    -0.00479664,
     -0.043298274,  -0.05090505,    0.029398112,
      0.011642447,   0.04183789,   -0.017999396,
      0.011026355,  0.049722955,    0.012025892,
      0.007331535,   0.01967245,  -0.0025621902,
      0.028765293, 0.0068937168,   0.0029231338,
    -0.0002095079,  0.032031864,     0.02518659,
     -0.032855466,   0.00758291, -0.00011585959,
     -0.034515556, -0.066151336,     0.03191643,
     -0.026680378,  0.017334407,   -0.025778342,
     -0.008119435, -0.002431255,   -0.009850676,
     -0.030725427,   0.08225489
  ],
  [
     0.018468002,  0.0054281265,  -0.017658807,
     0.013859263,    0.05341865,   0.041700866,
     0.031085702,   -0.06442814,   0.010460507,
  

## Truncating embeddings

The `text-embedding-004` model also supports lower embedding dimensions. Specify `outputDimensionality` to truncate the output.


In [12]:
const trucated_embeddings = await ai.models.embedContent({
  model: MODEL_ID,
  contents: ["Hello world"],
  config: {
    outputDimensionality: 10,
  },
});
console.log("Truncated Text Embeddings:", trucated_embeddings.embeddings?.[0].values);
console.log("Truncated Text Embeddings Length:", trucated_embeddings.embeddings?.[0].values?.length);


Truncated Text Embeddings: [
   0.013168517,   -0.00871193,
  -0.046782672, 0.00069969177,
  -0.009518872,  -0.008720178,
    0.06010358,   0.024755737,
   0.026053527,   0.054356426
]
Truncated Text Embeddings Length: 10


## Specify `taskType`

For details on how to call `embedContent`, check out the [Embeddings API reference](https://ai.google.dev/api/embeddings#method:-models.embedcontent), including the section on `taskType`.


In [13]:
const with_no_task_type = await ai.models.embedContent({
  model: MODEL_ID,
  contents: ["Hello world"],
});
console.log("With No Task Type Text Embeddings:", with_no_task_type.embeddings?.[0].values?.slice(0, 50), "...TRIMMED");

const with_task_type = await ai.models.embedContent({
  model: MODEL_ID,
  contents: ["Hello world"],
  config: {
    taskType: "retrieval_document",
  },
});
console.log("With Task Type Text Embeddings:", with_task_type.embeddings?.[0].values?.slice(0, 50), "...TRIMMED");


With No Task Type Text Embeddings: [
    0.013168517,   -0.00871193,  -0.046782672,
  0.00069969177,  -0.009518872,  -0.008720178,
     0.06010358,   0.024755737,   0.026053527,
    0.054356426,   -0.03793384, -0.0014235445,
    0.030605137,  -0.015512642,  -0.012904964,
    -0.02880739,  -0.007819577,   0.012152762,
     -0.1139952,   0.010654231,   0.005365246,
   -0.001178891,  -0.029781109,  -0.060107403,
   -0.015272871, -0.0036046242,   0.006147686,
    0.031175768,   0.021421982,    0.03710434,
   -0.037202735,   0.046146937,   0.002196372,
   -0.031793054,   0.009660255,   0.012500477,
     -0.0509635,     0.0211728,   0.014332891,
   -0.057802226,  -0.027034516,    0.03680537,
   0.0016361808,     0.0085209,    0.04331588,
   -0.032519087,   0.018076202, -0.0031592466,
   0.0045996527, -0.0063372543
] ...TRIMMED
With Task Type Text Embeddings: [
   0.023399517,  -0.008547142, -0.052534223,
  -0.012143115,  0.0055042417, -0.001007702,
   0.028462889,   0.037563253, -0.023794618

## Learning more

Check out these examples in the Cookbook to learn more about what you can do with embeddings:

* [Search Reranking](../examples/Search_reranking_using_embeddings.ipynb): Use embeddings from the Gemini API to rerank search results from Wikipedia.

* [Anomaly detection with embeddings](../examples/Anomaly_detection_with_embeddings.ipynb): Use embeddings from the Gemini API to detect potential outliers in your dataset.

* [Train a text classifier](../examples/Classify_text_with_embeddings.ipynb): Use embeddings from the Gemini API to train a model that can classify different types of newsgroup posts based on the topic.

* Embeddings have many applications in Vector Databases, too. Check out this [example with Chroma DB](../examples/chromadb/Vectordb_with_chroma.ipynb).

You can learn more about embeddings in general on ai.google.dev in the [embeddings guide](https://ai.google.dev/docs/embeddings_guide)

* You can find additional code examples with the JS SDK [here](https://ai.google.dev/gemini-api/docs/quickstart#javascript).

* You can also find more details in the API Reference for [embedContent](https://ai.google.dev/api/rest/v1/models/embedContent) and [batchEmbedContents](https://ai.google.dev/api/rest/v1/models/batchEmbedContents).